In [49]:
import sympy as sp
from sympy import symbols, sin, cos, Matrix, Eq, Rational, floor, sqrt
from sympy import simplify, factorial, pi, binomial, factor, expand, collect
from sympy.functions.special.tensor_functions import KroneckerDelta
from sympy import init_printing
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pandas as pd
from sympy import latex
from scipy.optimize import curve_fit
from IPython.display import display, Math

## Oblateness as a function of angluar velocity:

We have used f as an oblateness factor in other places, which is the effective oblateness of the star, agnostic of its rotation rate or other factors. However, we would like to know if we can find the oblateness of the star as a function of its rotation rate. 


According to [Bouvier et al. 2013](https://arxiv.org/pdf/1307.2891.pdf), the radius of a star can be written as a function of $\omega$ using the following relation (according to the Roche model for stellar equipotential surfaces):

$$R_{eq} = \frac{3 R_{polar} \cos{\left(\frac{\operatorname{arccos}{\left(\omega \right)}}{3} + \frac{\pi}{3} \right)}}{\omega}$$

b is defined as $\frac{R_{polar}}{R_{eq}}$, so we can write f as a function of omega as follows:

In [14]:
omega = sp.symbols(
    "omega"
    ,real=True,positive=True)
f_omega = sp.simplify((omega/3) / sin((sp.asin(omega))/3))
display(Math(r'f = ' + latex(1 - f_omega)))

<IPython.core.display.Math object>

## Von Zeipel's Law for an oblate star:

In [15]:
x0, y0, z0= sp.symbols("x0 y0 z0",real=True,positive=True) #can switch positive=True off, but sympy simplifies better
Omega, R, R_perp, G, M, beta, T_pole, g_pole = sp.symbols(
    "Omega R R_perp G M beta T_pole g_pole"
    ,real=True,positive=True)
R = sqrt(x0**2+y0**2+z0**2)
R_perp = sqrt(x0**2+z0**2)
R_vec = Matrix([x0/R,y0/R,z0/R])
R_perpvec = Matrix([x0/R_perp,0,  z0/R_perp])
g_vec = -(G*M/R**2)*R_vec + Omega**2*R_perp*R_perpvec
display(Math(r'\vec{g} = '+latex(g_vec)))

<IPython.core.display.Math object>

Again, $\Omega$, the angular rotational velocity, can be written as a dimensionless variable $\omega$ based on the prescription ([from Lara & Rieutord 2011](https://arxiv.org/abs/1109.3038)): $$ \omega = \Omega \sqrt{\frac{R_{eq}^3}{GM}}$$

In [28]:
R_eq, omega, b = sp.symbols("R_eq, omega, b",positive=True,real=True)
g_vec = g_vec.subs([(Omega,omega*sqrt(G*M/R_eq**3))])
display(Math(r'\vec{g} = ' + latex(g_vec)))

<IPython.core.display.Math object>

Writing this into Von Zeipel's Law:

In [29]:
g_pole = g_vec.subs([(x0,0),(y0,R_eq*b),(z0,0)])
T = T_pole * (g_vec.norm()/g_pole.norm())**beta
display(Math('T = ' + latex(T)))

<IPython.core.display.Math object>

In [50]:
g_pole.norm()

G*M/(R_eq**2*b**2)

In [51]:
T_simp = (T_pole * (simplify(g_vec.norm().subs([(z0,sqrt(R_eq**2 - x0**2 - (y0/b)**2))]))/g_pole.norm())**beta)
display(Math('T = ' + latex(T_simp)))

<IPython.core.display.Math object>

Assuming a spherical star, the equation for the star's photosphere would be: 
$$ x_0^2 + y_0^2 +z_0^2 = R_{eq}^2$$

From [Barnes et al. 2009](https://arxiv.org/pdf/0909.1752.pdf):

However, the star is actually oblate, so let's add the oblateness factor f in:

$$ x_0^2 + \left(\frac{y_0}{1-f}\right)^2 +z_0^2 = R_{eq}^2$$

In addition, the star could be angled by a stellar inclination of $\varphi$.

The rotation matrix for this is:

In [66]:
varphi = sp.symbols('varphi', real=True, positive=True)
rot = Matrix([[1,0,0],[0,cos(varphi),sin(varphi)],[0,-sin(varphi),cos(varphi)]])
display(Math(r'\vec{x_0} = ' + latex(rot)+r'\vec{x}'))

<IPython.core.display.Math object>

When we plug in the "true" coordinates x, y, and z into this rotation matrix we get:

In [67]:
x, y, z = sp.symbols('x, y, z', real=True,positive=True) #can switch positive=True off, but sympy simplifies better
x_vec = Matrix([x,y,z])
x0_vec = Matrix([rot.row(0).dot(x_vec),rot.row(1).dot(x_vec),rot.row(2).dot(x_vec)])
display(Math(r'\begin{bmatrix}x_0\\y_0\\z_0\end{bmatrix} = ' + latex(x0_vec)))

<IPython.core.display.Math object>

In the simple spherical case the surface of the star can be modeled as a function of x and y alone:  $$z(x, y) = \sqrt{R_{eq}^2 - x^2 -y^2}$$.

Due to the oblateness, however, things are more complicated, and z(x, y) must also include $\varphi$ and $f$. This can be found (in the same way as Barnes et al. 2009), by solving the quadratic equation:

In [90]:
x, y, z= sp.symbols("x y z",real=True,positive=True) #can switch positive=True off, but sympy simplifies better

z_soln = sp.solvers.solve(x**2 + ((y*cos(varphi) + z*sin(varphi))/b)**2 + (-y*sin(varphi) + z*cos(varphi))**2 - R_eq**2, z)[0]
z_soln = sp.simplify(z_soln)
display(Math(r'z = ' + latex(z_soln)))

<IPython.core.display.Math object>

In [91]:
z_soln.subs([(varphi,pi/2)])

-b*sqrt(R_eq**2 - x**2 - y**2)

In [92]:
T_simp

T_pole*(b*sqrt((R_eq**6*b**8*y0**2 + b**2*x0**2*(R_eq**3*b**3 - omega**2*(b**2*(R_eq**2 + y0**2) - y0**2)**(3/2))**2 + (R_eq**3*b**3 - omega**2*(b**2*(R_eq**2 + y0**2) - y0**2)**(3/2))**2*(b**2*(R_eq**2 - x0**2) - y0**2))/(b**2*(R_eq**2 + y0**2) - y0**2)**3)/R_eq)**beta

Let's make the simplification of $R_{eq} = 1$ and $\varphi = \pi/2$ early because the math gets heinous from here on out. Also, let's substitute $x$ and $y$ for $x_0, y_0, z_0$. 

In [93]:
T_rot = T_simp.subs([(x0,x),(y0,(y*cos(varphi) + (z_soln)*sin(varphi))),(z0,(-y*sin(varphi) + (z_soln)*cos(varphi)))])
T_rot = T_rot.subs([(R_eq,1), (varphi, pi/2)])
display(Math(r'T(x,y,\varphi) = ' + latex(T_rot)))

<IPython.core.display.Math object>

In [94]:
T_rot.subs([(b, 1)])

T_pole*(x**2*(1 - omega**2)**2 - x**2 + y**2*(1 - omega**2)**2 - y**2 + 1)**(beta/2)

In [86]:
alpha = sp.symbols("alpha",real=True)
T_rot_simp = simplify(T_rot.subs([(1-x**2-y**2, alpha)]))

In [87]:
T_rot_simp

T_pole*(b**2*sqrt((-alpha*b**2 - x**2*(-omega**2*(alpha*b**2 - alpha + 1)**(3/2) + 1)**2 + (-omega**2*(alpha*b**2 - alpha + 1)**(3/2) + 1)**2*(alpha + x**2 - 1))/(-alpha*b**2 + alpha - 1)**3))**beta

In [88]:
T_rot_simp = collect(T_rot_simp, (-omega**2*sqrt(alpha*b**2 - alpha + 1)**3 + 1)**2)
T_rot_simp

T_pole*(b**2*sqrt((-alpha*b**2 + (alpha - 1)*(-omega**2*(alpha*b**2 - alpha + 1)**(3/2) + 1)**2)/(-alpha*b**2 + alpha - 1)**3))**beta

In [104]:
Tpole = T_rot_simp.evalf(subs={alpha: 1, b:0.98,beta:0.22, omega:0.23, T_pole:7600})

In [101]:
T_eq = T_rot_simp.evalf(subs={alpha: 0, b:0.98,beta:0.22, omega:0.23, T_pole:7600})

In [102]:
T_eq

7443.20789966145

In [105]:
T_eq/Tpole

0.979369460481770